In [44]:
pip install pandas dash

In [45]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [46]:
spacex_df = pd.read_csv("combined_dataset.csv")


In [47]:
spacex_df['Payload mass'] = spacex_df['Payload mass'].str.replace(',', '').str.replace(' kg', '')
spacex_df['Payload mass'] = pd.to_numeric(spacex_df['Payload mass'], errors='coerce')

In [48]:
max_payload = int(spacex_df['Payload mass'].max())
min_payload = spacex_df['Payload mass'].min()

In [49]:
app = dash.Dash(__name__)

In [50]:
launch_sites = spacex_df['Launch site'].unique()

In [51]:
dropdown_options = [{'label': 'All' , 'value': 'All'}]
dropdown_options.extend([{'label': site , 'value': site} for site in launch_sites])

In [52]:
marks = {i: str(i) for i in range(0, max_payload, 1000)}

In [53]:
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                             options=dropdown_options,
                                             value = 'All',
                                             placeholder = "Select a Launch Site here",
                                             searchable= True
                                             ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0,max=max_payload, step =1000,
                                                marks=marks,
                                                value= [min_payload, max_payload]
                                            ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),


                                ])

In [54]:
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def update_pie_chart(entered_site):
    if entered_site == 'All':
        # If 'ALL' is selected, calculate total successful launches count for all sites
        total_success = spacex_df[spacex_df['Booster landing'] == 1]['Booster landing'].count()
        total_failed = spacex_df[spacex_df['Booster landing'] == 0]['Booster landing'].count()
        labels = ['Successful', 'Failed']
        values = [total_success, total_failed]
        title = 'Total Successful vs. Failed Launches'
    else:
        # If a specific site is selected, calculate success vs. failed counts for that site
        site_df = spacex_df[spacex_df['Launch site'] == entered_site]
        site_success = site_df[site_df['Booster landing'] == 1]['Booster landing'].count()
        site_failed = site_df[site_df['Booster landing'] == 0]['Booster landing'].count()
        labels = ['Successful', 'Failed']
        values = [site_success, site_failed]
        title = f'Success vs. Failed Launches for {entered_site}'

    # Create pie chart figure
    fig = px.pie(names=labels, values=values, title=title)
    return fig

In [55]:
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id='payload-slider', component_property='value')]
)
def update_scatter_chart(selected_site, selected_payload_range):
    if selected_site == 'All':
        # Filter spacex_df based on selected payload range
        filtered_df = spacex_df[(spacex_df['Payload mass'] >= selected_payload_range[0]) &
                                (spacex_df['Payload mass'] <= selected_payload_range[1])]

        filtered_df = filtered_df.sort_values(by='Payload mass')
        # Create scatter plot with all sites
        fig = px.scatter(filtered_df, x='Payload mass', y='Booster landing', color='Version Booster',
                         title='Payload Mass vs. Success/Failure (All Sites)')
    else:
        # Filter spacex_df for the selected launch site and payload range
        filtered_df = spacex_df[(spacex_df['Launch site'] == selected_site) &
                                (spacex_df['Payload mass'] >= selected_payload_range[0]) &
                                (spacex_df['Payload mass'] <= selected_payload_range[1])]

        filtered_df = filtered_df.sort_values(by='Payload mass')
        # Create scatter plot for the selected launch site
        fig = px.scatter(filtered_df, x='Payload mass', y='Booster landing', color='Version Booster',
                         title=f'Payload Mass vs. Success/Failure ({selected_site})')

    print(filtered_df)

    return fig


In [56]:
# Run the app
if __name__ == '__main__':
    app.run_server()

<IPython.core.display.Javascript object>